A notebook to illustrate/test `kmod.mctest.DS_FSSD`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import kmod
import kgof
import kgof.goftest as gof
# submodules
from kmod import data, density, kernel, util, plot
from kmod import mctest as mct
import matplotlib
import matplotlib.pyplot as plt
import autograd.numpy as np
import scipy.stats as stats

In [ ]:
plot.set_default_matplotlib_options()
# # font options
# font = {
#     #'family' : 'normal',
#     #'weight' : 'bold',
#     'size'   : 18
# }

# plt.rc('font', **font)
# plt.rc('lines', linewidth=2)
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42

## Simple problem: 1D normal distributions

Data are drawn from $r=\mathcal{N}(\mu_r, \sigma_r^2)$. Two models: $p=\mathcal{N}(\mu_p, \sigma_p^2)$ and $q=\mathcal{N}(\mu_q, \sigma_q^2)$.

In [ ]:
# Data generating distribution
seed = 21

mu_r = 0
var_r = 1
r = density.IsotropicNormal(np.array([mu_r]), var_r)
ds_r = r.get_datasource()
# sample data
n = 500
dat = ds_r.sample(n, seed=seed+1)
X = dat.data()

In [ ]:
def visual_test_1dGauss(X, mu_p, var_p, mu_q, var_q, J=5):
    """
    Assume that both models p,q are 1D normal distributions.
    Plot p, q, and the data and test with the FSSD-based model 
    comparison test.
    
    :param X: data matrix
    :param mu_p: mean of the model p
    :param var_p: variance of the model p
    :param mu_q: mean of the model q
    :param var_q: variance of the model q
    
    :returns the test object
    """
    p = density.IsotropicNormal(np.array([mu_p]), var_p)
    q = density.IsotropicNormal(np.array([mu_q]), var_q)
    
    # kernel
    med = util.meddistance(X, subsample=1000)
    k = kernel.KGauss(sigma2=med**2/2.0)
    l = k

    # numbers of test locations
    Jp = J
    Jq = Jp
    # test locations
#     locs = util.fit_gaussian_draw(X, Jp+Jq, seed=seed+1)
#     V = locs[:Jp, :]
#     W = locs[Jp:, :]k
    locs = util.fit_gaussian_draw(X, Jp, seed=seed+1)
    V = locs
    W = V
    assert W.shape[0] == Jq
    
    # Plot p, q, data
    min_mean = min(mu_r, mu_p, mu_q)
    max_mean = max(mu_r, mu_p, mu_q)
    max_sd = max(var_r, var_p, var_q)**0.5

    dom = np.linspace(min_mean-2*max_sd, max_mean+2*max_sd, 200)
    den_p = np.exp(p.log_normalized_den(dom[:, np.newaxis]))
    den_q = np.exp(q.log_normalized_den(dom[:, np.newaxis]))

    plt.figure(figsize=(10, 5))
    plt.hist(X, bins=20, normed=True, label='Data', color='k')
    plt.plot(dom, den_p, 'r-', label='p')
    plt.plot(dom, den_q, 'b-', label='q')
    plt.title('H1: q fits better than p')
    plt.legend(loc='best')

    # test
    alpha = 0.01
    mcfssd = mct.DC_FSSD(p, q, k, l, V, W, alpha=alpha)
    test_result = mcfssd.perform_test(dat)
    display(test_result)
    return mcfssd

In [ ]:
# two competing models. H0: FSSD^2(p) <= FSSD^2(q) i.e., p is better than q.
mu_p, var_p = 1.5, 2
mu_q, var_q = 0.5, 1
mcfssd = visual_test_1dGauss(X, mu_p, var_p, mu_q, var_q, J=5)

In [ ]:
mcfssd.get_H1_mean_variance(dat)

In [ ]:
import ipywidgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display
import ipywidgets as widgets

mup_slide = ipywidgets.FloatSlider(value=1, min=-3, max=3, step=0.5)
muq_slide = ipywidgets.FloatSlider(value=0.5, min=-3, max=3.0, step=0.5)
vs = interact(visual_test_1dGauss, X=fixed(X), mu_p=mup_slide, var_p=(1e-4, 5, 0.5),
                mu_q=muq_slide, var_q=(1e-4, 5, 0.5))
display(vs)

----------------

## Stein witness functions

In [ ]:
# models
p = density.IsotropicNormal(mean=np.array([0]), variance=1)
q = density.IsotropicNormal(mean=np.array([6]), variance=1)
r = density.IsoGaussianMixture(
    means=np.array([[0.0, 6]]).T,
    variances=np.array([1, 1]),
    pmix=[0.5, 0.5]
)
dsr = r.get_datasource()
rdat = dsr.sample(n=1000, seed=58)
med = util.meddistance(rdat.data(), subsample=1000)
# k = kernel.KGauss(sigma2=med**2/2.0)
k = kernel.KGauss(sigma2=1.0)

In [ ]:
wit_pr = gof.SteinWitness(p, k, rdat)
wit_qr = gof.SteinWitness(q, k, rdat)
dom = np.linspace(-5, 10, 200)
wit_pr_evals = wit_pr(dom[:, np.newaxis])
wit_qr_evals = wit_qr(dom[:, np.newaxis])
diff_wit2 = wit_pr_evals**2 - wit_qr_evals**2

In [ ]:
# visual_test_1dGauss(rdat.data(), mu_p, var_p, mu_q, var_q)

# plot the densities
dom = np.linspace(-6, 11, 200)[:, np.newaxis]
denp = np.exp(p.log_normalized_den(dom))
denq = np.exp(q.log_normalized_den(dom))
denr = np.exp(r.log_normalized_den(dom))

plt.figure(figsize=(8, 5))
density_lw = 2
plt.plot(dom, denp, 'r-', linewidth=density_lw, label='p')
plt.plot(dom, denq, 'b-', linewidth=density_lw, label='q')
plt.plot(dom, denr, 'k-', linewidth=density_lw, label='r')

# plot the witness functions
wit_scale = 0.4
diff_wit2_scale = 2
wit_lw = 2
diff_wit2_lw = 4
plt.plot(dom, wit_scale*wit_pr_evals, 'r--', linewidth=wit_lw, label='swit p,r')
plt.plot(dom, wit_scale*wit_qr_evals, 'b--', linewidth=wit_lw, label='swit q,r')
plt.plot(dom, diff_wit2_scale*diff_wit2, 'm-', 
         linewidth=diff_wit2_lw, label='swit$^2$(p,r)-swit$^2$(q,r)')
plt.legend(loc='lower left', bbox_to_anchor=(1., 0.))
plt.savefig('dcfssd_stein_diff.pdf', bbox_inches='tight')

## Optimize test locations and the Gaussian width

In [ ]:
# r = N(0, 1)
mu_p, var_p = 1, 1.0
mu_q, var_q = 0.3, 1.0
p = density.IsotropicNormal(mean=np.array([mu_p]), variance=var_p)
q = density.IsotropicNormal(mean=np.array([mu_q]), variance=var_q)

n = 500
seed = 8
datr = ds_r.sample(n=n, seed=seed)
# split the data into training/test sets
datr_tr, datr_te = datr.split_tr_te(tr_proportion=0.5)

In [ ]:
# def diversity_obj(gwidth2, V):
#     """
#     An experimental hacky function to promote diversity of the test locations.
#     This is experimental.
#     """
#     J, d = V.shape
#     if J==1:
#         return 0
#     scale = 1.0/(d*J*(J-1))
#     return -scale*(np.sum(np.sum(V, axis=0)**2) - np.sum(np.sum(V**2, axis=1)))

In [ ]:
# initialize parameters to optimize
med = util.meddistance(rdat.data(), subsample=1000)
gwidth0 = med**2

# J = number of test locations
J = 5
Z_tr = datr_tr.data()
V0 = util.subsample_rows(Z_tr, J, seed=seed+2)

## Optimize
opt_options = {
    'reg': 1e-2,
    'max_iter': 100,
    'tol_fun': 1e-6,
    'locs_bounds_frac': 100,
    'gwidth_lb': 1e-1,
    'gwidth_ub': 6**2,
#     'added_obj': diversity_obj,
}
V_opt, gw_opt, opt_info = mct.DC_GaussFSSD.optimize_power_criterion(p, q, rdat, V0, gwidth0, **opt_options)


In [ ]:
display(opt_info)

In [ ]:
# test
alpha = 0.01
dcfssd_opt = mct.DC_GaussFSSD(p, q, gw_opt, gw_opt, V_opt, V_opt, alpha=alpha)
dcfssd_opt.perform_test(datr_te)

In [ ]:
dom = np.linspace(-3, 5, 200)[:, np.newaxis]
pden = np.exp(p.log_normalized_den(dom))
qden = np.exp(q.log_normalized_den(dom))

# plot the data and the learned locations
plt.figure(figsize=(8, 4))
plt.plot(dom, pden, 'r-', label='p')
plt.plot(dom, qden, 'b-', label='q')
plt.hist(Z_tr, color='k', alpha=0.8, normed=True, label='R sample')
for v in V_opt:
    plt.plot(v[0], 0, '^', color='magenta', markersize=40)
    
plt.title('H1: q is closer to R')
plt.legend(loc='lower left', bbox_to_anchor=(1, 0))